In [ ]:
import tensorflow as tf
from keras.layers import Conv2D, Flatten, UpSampling2D, InputLayer
from keras.models import Sequential
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import numpy as np
import cv2 as cv

In [ ]:
# Get the luminance of the image aka the image in black and white
def getLFromLab(image_loaded):
    L_image = cv.cvtColor(np.float32(
        1.0/255 * image_loaded), cv.COLOR_RGB2LAB)[:, :, 0]
    L_image = L_image.reshape(1, width, height, 1)
    return L_image

In [ ]:
# Building the neural network
def buildSimpleNN():
    model = Sequential()
    model.add(InputLayer(input_shape=(None, None, 1)))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

    model.compile(optimizer='rmsprop', loss='mse')

    return model

In [ ]:
# Training the neural network
def trainModel(X, Y, model):
    print('Training model...')
    model.fit(x=X[:amount], y=Y[:amount], batch_size=2, epochs=30, verbose=1)
    print('Model trained.')